In [2]:
%load_ext cython

In [11]:
%%cython
#cython: language_level=3

cimport cython
cimport numpy as np
import numpy as np

# Assuming string->string change, you can reuse this bit.
ctypedef str (*string_transform)(str)

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
cdef np.ndarray transform_memoryview(str[:] arr, string_transform f):
    cdef str[:] result_view
    cdef Py_ssize_t i
    cdef str s
    result = np.ndarray((len(arr),), dtype=object)
    result_view = result
    for i in range(len(arr)):
        s = arr[i]
        result_view[i] = f(s)
    return result


# And just make more of these. Notice this is a C function (cdef) with type annotations.
cdef str _replace_then_upper(str s):
    return s.replace("l", "").upper()[1:-1]

# This deliberately compiles the two cdef functions together inside Cython, in the hopes the compiler will combine them.
def replace_then_upper(str[:] arr) -> np.ndarray:
    return transform_memoryview(arr, _replace_then_upper)

In [8]:
import pandas as pd

STRINGS = ["{} hello world how are you".format(i) for i in range(1_000_000)]
SERIES = pd.Series(STRINGS)

In [14]:
%time s1 = pd.Series(replace_then_upper(SERIES.values))
%time s2 = SERIES.apply(lambda s: s.replace("l", "").upper()[1:-1])
all(s1.values == s2.values)

CPU times: user 285 ms, sys: 24.9 ms, total: 310 ms
Wall time: 313 ms
CPU times: user 446 ms, sys: 33.1 ms, total: 479 ms
Wall time: 484 ms


True